<a href="https://www.kaggle.com/code/faisalalbasu/spectrogram-windows-test?scriptVersionId=143811693" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import scipy
import scipy.signal
import matplotlib as mpl
import matplotlib.pyplot as plt
import random
import seaborn as sns
from matplotlib.colors import ListedColormap

from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing as prep
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score as acc
from sklearn.metrics import balanced_accuracy_score as bal_acc
from sklearn.metrics import f1_score as f1
import sklearn.metrics
import scipy.stats as stats
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

!pip install dtreeviz
from dtreeviz.trees import *
import dtreeviz

try:
    import joblib
except:
    !pip install joblib
finally:
    import joblib

# plt.style.use('seaborn')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# small sampled data
# !wget 'https://raw.githubusercontent.com/albas99/Fourier-Transform/master/sample%20analysis.csv'

In [ ]:
# small sample with diagnoses
# !wget 'https://raw.githubusercontent.com/albas99/Fourier-Transform/master/sample_analysis_diagnoses.csv'

In [ ]:
# full data for maximimum signals
!wget 'https://raw.githubusercontent.com/albas99/Fourier-Transform/master/maximum.csv'

In [ ]:
# full data with diagnoses
!wget 'https://raw.githubusercontent.com/albas99/Fourier-Transform/master/maximum_diagnoses.csv'

In [ ]:
max_erg = pd.read_csv('/kaggle/working/maximum.csv', sep = ',', index_col = 'Time,ms')
max_erg.head()

In [ ]:
diag_max_erg = pd.read_csv('/kaggle/working/maximum_diagnoses.csv', sep = ',')
diag_max_erg

In [ ]:
diag_max_erg.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
# data = pd.read_csv('/kaggle/working/sample analysis.csv', sep = ',', index_col='Time,ms')
# data

In [ ]:
# diagdata = pd.read_csv('/kaggle/working/sample_analysis_diagnoses.csv', sep = ',')
# diagdata

In [ ]:
# diagdata.drop('Unnamed: 0', axis = 1, inplace = True)
# diagdata

In [ ]:
NFFTS = [128, 64, 32, 16, 8, 4]
overlaps = [64, 32, 16, 8, 4, 2]
windows = ['hann', 'hamming', 'blackman', 'tukey', 'boxcar', 'bartlett', 'blackmanharris', 'taylor']

In [ ]:
def plot_spectrogram(sig: str, nfft, noverlap, colormap):
    time_step = 0.005 # in seconds
    sample_freq = 1 / time_step
    sig = np.array(max_erg[sig])
    
    assert sig.ndim  == 1
    assert nfft <= len(sig)
    
#     window = np.blackman(nfft)
    window = signal.get_window('taylor', nfft)
    
    Sxx, freqs, bins, im = plt.specgram(sig, Fs = sample_freq, NFFT = nfft, noverlap = noverlap, cmap = colormap, window = window)
    plt.colorbar().set_label('Power')
    plt.xlabel('Time')
    plt.ylabel('Frequency')
    plt.show()
    return Sxx

In [ ]:
# a = plot_spectrogram('3', 32, 16, 'jet')
# a

In [ ]:
def load_signal(sig):
    assert type(sig) is str, f"{type(sig)}"
    signal_no = np.array(max_erg[sig])
    return signal_no

In [ ]:
def compute_spectrogram(signal_no, nperseg, noverlap, window):
    time_step = 0.005
    sample_freq = 1 / time_step
    freqs, times, Sxx = scipy.signal.spectrogram(signal_no, fs = sample_freq, nperseg = nperseg, noverlap = noverlap, window = window)
    
    return freqs, times, Sxx

In [ ]:
# def divide_quadrants(Sxx):
#     num_t, num_f = Sxx.shape
#     quadrant1 = Sxx[:num_t//2, :num_f//2]  # Top-left quadrant
#     quadrant2 = Sxx[:num_t//2, num_f//2:]  # Top-right quadrant
#     quadrant3 = Sxx[num_t//2:, :num_f//2]  # Bottom-left quadrant
#     quadrant4 = Sxx[num_t//2:, num_f//2:]  # Bottom-right quadrant
#     print(quadrant1, quadrant2, quadrant3, quadrant4)
#     return quadrant1, quadrant2, quadrant3, quadrant4

In [ ]:
# a = np.linspace(0, 50)
# b = np.reshape(a, (5, 10))
# c, d = b.shape
# b[:c//2]

In [ ]:
def compute_features(sig, nperseg, noverlap, window):
    freqs, times, Sxx = compute_spectrogram(signal_no, nperseg, noverlap, window)
    spectrum = Sxx.flatten()
    bmin = np.min(spectrum)
    bmedian = np.median(spectrum)
    bmax = np.max(spectrum)
    bmean = np.mean(spectrum)
    return bmin, bmedian, bmax, bmean

I've tried dividing the spectrogram into quadrants but I keep getting issue where some of the quadrants would return empty arrays, so I've given up on that for now.

In [ ]:
# def get_features(sig, nperseg, noverlap, window):
#     signal_no = load_signal(sig)
#     freqs, times, Sxx = compute_spectrogram(signal_no, nperseg, noverlap, window)
#     quadrant1, quadrant2, quadrant3, quadrant4 = divide_quadrants(Sxx)
#     q1bmin, q1bmedian, q1bmax, q1bmean = compute_features(quadrant1)
#     q2bmin, q2bmedian, q2bmax, q2bmean = compute_features(quadrant2)
#     q3bmin, q3bmedian, q3bmax, q3bmean = compute_features(quadrant3)
#     q4bmin, q4bmedian, q4bmax, q4bmean = compute_features(quadrant4)
#     return q1bmin, q1bmedian, q1bmax, q1bmean, q2bmin, q2bmedian, q2bmax, q2bmean, q3bmin, q3bmedian, q3bmax, q3bmean, q4bmin, q4bmedian, q4bmax, q4bmean

In [ ]:
# def get_features(sig, nperseg, noverlap, window):
#     freqs, times, Sxx = compute_spectrogram(sig = sig, nperseg = nperseg, noverlap = noverlap, window = window)
#     num_t, num_f = Sxx.shape
#     quadrant1 = Sxx[:num_t//2, :num_f//2]  # Top-left quadrant
#     quadrant2 = Sxx[:num_t//2, num_f//2:]  # Top-right quadrant
#     quadrant3 = Sxx[num_t//2:, :num_f//2]  # Bottom-left quadrant
#     quadrant4 = Sxx[num_t//2:, num_f//2:]  # Bottom-right quadrant
# #     print(f"q1: {quadrant1}, q2: {quadrant2}, q3: {quadrant3}, q4: {quadrant4}")

#      # Check if any quadrant is empty
#     assert quadrant1.size > 0, f"Top-left quadrant of {sig} is empty."
#     assert quadrant2.size > 0, f"Top-right quadrant of {sig} is empty."
#     assert quadrant3.size > 0, f"Bottom-left quadrant of {sig} is empty."
#     assert quadrant4.size > 0, f"Bottom-right quadrant of {sig} is empty."
    
#     q1spectrum = quadrant1.flatten()
#     q2spectrum = quadrant2.flatten()
#     q3spectrum = quadrant3.flatten()
#     q4spectrum = quadrant4.flatten()
    
#     # Check if any spectrum is empty after flattening
# #     assert q1spectrum.size > 0, "Top-left spectrum is empty."
# #     assert q2spectrum.size > 0, "Top-right spectrum is empty."
# #     assert q3spectrum.size > 0, "Bottom-left spectrum is empty."
# #     assert q4spectrum.size > 0, "Bottom-right spectrum is empty."
# #     print(f"q1s: {q1spectrum}, q2s: {q2spectrum}, q3s: {q3spectrum}, q4s: {q4spectrum}")

# #     if len(q1spectrum) == 0:
# #         print(f"empty array at {sig}, {q1spectrum}, {quadrant1}")
# #         return

#   # Compute features for each quadrant
#     q1bmin, q1bmedian, q1bmax, q1bmean = np.min(q1spectrum), np.median(q1spectrum), np.max(q1spectrum), np.mean(q1spectrum)
#     q2bmin, q2bmedian, q2bmax, q2bmean = np.min(q2spectrum), np.median(q2spectrum), np.max(q2spectrum), np.mean(q2spectrum)
#     q3bmin, q3bmedian, q3bmax, q3bmean = np.min(q3spectrum), np.median(q3spectrum), np.max(q3spectrum), np.mean(q3spectrum)
#     q4bmin, q4bmedian, q4bmax, q4bmean = np.min(q4spectrum), np.median(q4spectrum), np.max(q4spectrum), np.mean(q4spectrum)
    
# #     print(f"{sig} q1s: {q1spectrum}, q2s: {q2spectrum}, q3s: {q3spectrum}, q4s: {q4spectrum}")
# #     print(f"q1s shape: {q1spectrum.shape}, q2s shape: {q2spectrum.shape}, q3s shape: {q3spectrum.shape}, q4s shape: {q4spectrum.shape}")

    
#     return q1bmin, q1bmedian, q1bmax, q1bmean, q2bmin, q2bmedian, q2bmax, q2bmean, q3bmin, q3bmedian, q3bmax, q3bmean, q4bmin, q4bmedian, q4bmax, q4bmean

In [ ]:
# for column in max_erg.columns:
#     if len(column) == 0:
#         print(f"empty array at {column}")
#     print(get_features(column, 32, 16, 'tukey'))

# get_features('6.1', 32, 16, 'tukey')

In [ ]:
# def get_parameters():
#     size, overlap, name = None, None, None
#     for n in NFFTS:
#         size = n
#     for o in overlaps:
#         overlap = o
#     for w in windows:
#         name = w
#     # use random.choice instead of loop
#     # then loop through the signals instead
#     return size, overlap, name

# print(get_parameters())

In [ ]:
import random
def get_parameters():
    window_type = random.choice(windows)
    
    window_size = random.choice(NFFTS)

    overlap = random.choice(overlaps)
    
    while overlap >= window_size:
        overlap = random.choice(overlaps)

#     if overlap >= window_size:
#         window_size = random.choice(NFFTS)
#         overlap = random.choice(overlaps)
#     else:
#         return window_size, overlap, window_type
#         print(window_size, overlap, window_type)


    
    return window_size, overlap, window_type

#     print(window_size, overlap, window_type)
    
# get_parameters()

In [ ]:
# Run after

In [ ]:
dfeatures = pd.DataFrame(columns = ['params', 'Bmin', 'Bmedian', 'Bmax', 'Bmean', 'pat_no'])
dfeatures

In [ ]:
max_erg.columns

In [ ]:
window_size, overlap, window_type = get_parameters()
for col in max_erg.columns:

    signal_no = load_signal(col)
#     freqs, times, Sxx = compute_spectrogram(signal_no, nperseg = window_size, noverlap = overlap, window = window_type)
    bmin, bmedian, bmax, bmean = compute_features(signal_no, window_size, overlap, window_type)

    # Create a dictionary with the features
    features_dict = {
        'params': f"{window_type}{window_size}{overlap}",
        'Bmin': bmin,
        'Bmedian': bmedian,
        'Bmax': bmax,
        'Bmean': bmean,
        'pat_no': col
    }

    features = pd.DataFrame(features_dict, index = range(0, len(features_dict)))

    dfeatures = pd.concat([dfeatures, features], ignore_index = True)

In [ ]:
dfeatures

In [ ]:
dfeatures.drop_duplicates(inplace = True)
dfeatures.reset_index(drop = True, inplace = True)

In [ ]:
# for col in max_erg.columns:
#     window_size, overlap, window_type = get_parameters()
#     signal_no = load_signal(col)
#     freqs, times, Sxx = compute_spectrogram(signal_no, nperseg = window_size, noverlap = overlap, window = window_type)
#     quadrant1, quadrant2, quadrant3, quadrant4 = divide_quadrants(Sxx)
#     q1bmin, q1bmedian, q1bmax, q1bmean = compute_features(quadrant1)
#     q2bmin, q2bmedian, q2bmax, q2bmean = compute_features(quadrant2)
#     q3bmin, q3bmedian, q3bmax, q3bmean = compute_features(quadrant3)
#     q4bmin, q4bmedian, q4bmax, q4bmean = compute_features(quadrant4)
    
#     features = {
#         'Params': f'{window_type}{window_size}{overlap}',
#         'Q1_Min': q1bmin,
#         'Q1_Median': q1bmedian,
#         'Q1_Max': q1bmax,
#         'Q1_Mean': q1bmean,
#         'Q2_Min': q2bmin,
#         'Q2_Median': q2bmedian,
#         'Q2_Max': q2bmax,
#         'Q2_Mean': q2bmean,
#         'Q3_Min': q3bmin,
#         'Q3_Median': q3bmedian,
#         'Q3_Max': q3bmax,
#         'Q3_Mean': q3bmean,
#         'Q4_Min': q4bmin,
#         'Q4_Median': q4bmedian,
#         'Q4_Max': q4bmax,
#         'Q4_Mean': q4bmean
#     }
    
#     dfeatures = dfeatures.append(features, ignore_index = True)

In [ ]:
# for _ in range(len(max_erg.columns)):
#     window_size, overlap, window_type = get_parameters()
#     for col in max_erg.columns:
        
#         q1bmin, q1bmedian, q1bmax, q1bmean, q2bmin, q2bmedian, q2bmax, q2bmean, q3bmin, q3bmedian, q3bmax, q3bmean, q4bmin, q4bmedian, q4bmax, q4bmean = get_features(str(col), window_size, overlap, window_type)
        
#         new_dict = {'params': f'{window_type}{window_size}{overlap}', 'q1bmin': q1bmin, 'q1bmedian': q1bmedian, 'q1bmax': q1bmax, 'q1bmean': q1bmax, 'q2bmin': q2bmin, 'q2bmedian': q2bmedian, 'q2bmax': q2bmax, 'q2bmean': q2bmax, 'q3bmin': q3bmin, 'q3bmedian': q3bmedian, 'q3bmax': q3bmax, 'q3bmean': q3bmax, 'q4bmin': q4bmin, 'q4bmedian': q4bmedian, 'q4bmax': q4bmax, 'q4bmean': q4bmax, 'pat_no': col}
#         new_features = pd.DataFrame(data = new_dict, index = range(0, len(new_dict)))
#     dfeatures = pd.concat([dfeatures, new_features], ignore_index = True)

In [ ]:
dfeatures

In [ ]:
max_erg.columns

In [ ]:
# # dfeatures_list = []
# for col in max_erg.columns:
#     window_size, overlap, window_type = get_parameters()
#     q1bmin, q1bmedian, q1bmax, q1bmean, q2bmin, q2bmedian, q2bmax, q2bmean, q3bmin, q3bmedian, q3bmax, q3bmean, q4bmin, q4bmedian, q4bmax, q4bmean = get_features(str(col), window_size, overlap, window_type)
        
#     new_dict = {'params': f'{window_type}{window_size}{overlap}', 'q1bmin': q1bmin, 'q1bmedian': q1bmedian, 'q1bmax': q1bmax, 'q1bmean': q1bmax, 'q2bmin': q2bmin, 'q2bmedian': q2bmedian, 'q2bmax': q2bmax, 'q2bmean': q2bmax, 'q3bmin': q3bmin, 'q3bmedian': q3bmedian, 'q3bmax': q3bmax, 'q3bmean': q3bmax, 'q4bmin': q4bmin, 'q4bmedian': q4bmedian, 'q4bmax': q4bmax, 'q4bmean': q4bmax, 'pat_no': col}
# new_features = pd.DataFrame(data = new_dict, index = range(0, len(new_dict)))
        
# dfeatures = pd.concat([dfeatuers, new_features], ignore_index = True)
    

In [ ]:
# new_features_list = []
# for col in data.columns:
#     print(col)
#     window_size, overlap, window_type = get_parameters()
#     bmin, bmedian, bmax, bmean = get_features(str(col), window_size, overlap, window_type)
#     new_dict = {'params': f'{window_type}{window_size}{overlap}', 'bmin': bmin, 'bmedian': bmedian, 'bmax': bmax, 'bmean': bmean, 'pat_no': col}
#     new_features = pd.DataFrame(data = new_dict, index = range(0, len(new_dict)))
#     new_features_list.append(new_features)
# dfeatures = pd.concat(new_features_list, ignore_index = True)
    
# # dfeatures

In [ ]:
# transposed_diagdata = diagdata.T
# transposed_diagdata.index.name = 'pat_no'
# transposed_diagdata.columns = ['targets']
# transposed_diagdata

In [ ]:
T_diagnoses = diag_max_erg.T
T_diagnoses.index.name = 'pat_no'
T_diagnoses.columns = ['diagnoses']
T_diagnoses

In [ ]:
dfeatures = dfeatures.merge(T_diagnoses, on = 'pat_no', how = 'left')

In [ ]:
dfeatures.head(30)

In [ ]:
diagnoses = dfeatures['diagnoses']
diagnoses

In [ ]:
sampled_features = dfeatures.drop(['pat_no', 'diagnoses', 'params'], axis = 1)

In [ ]:
sampled_features

### Label Encoding of Targets

In [ ]:
def encode_targets(to_encode):
    label = LabelEncoder()
    label.fit(to_encode)
    target_labels = label.classes_
    encoded_targets = label.transform(to_encode)
    
    return target_labels, encoded_targets

In [ ]:
features = sampled_features.columns
features

In [ ]:
def split_dataset(dataset, targets):
    random_state = 42
    X_train, X_val, y_train, y_val = train_test_split(dataset, encoded_targets, test_size = 0.4, random_state = random_state, stratify = targets)
    return X_train, X_val, y_train, y_val

In [ ]:
def train_model(X_set, y_set):
    max_depth = 7
    min_samples_split = 15
    max_leaf_nodes = 15
    clftree = DecisionTreeClassifier(max_depth = max_depth, min_samples_split = min_samples_split, max_leaf_nodes = max_leaf_nodes)
    clftree.fit(X_set, y_set)
    return clftree

In [ ]:
def cross_validation (X_set, y_set, model, cv_rule = StratifiedKFold(n_splits = 5)):
    scoring = {'ACC': 'accuracy', 'bal_ACC': 'balanced_accuracy'}
    model = train_model(X_set, y_set)
    scores = cross_validate(model,X_set, y_set,
                      scoring=scoring, cv=cv_rule )
    print('Cross-Validation error')
    DF_poly = pd.DataFrame(scores)
    display(DF_poly)
    print('\n')
    print(DF_poly.mean()[2:])
#     return scores

def test_model(X_set, y_set, model):
    y_predict = model.predict(X_set)
    return y_predict

def calculate_accuracies(y_true, y_pred):
    acc_score = acc(y_true, y_pred)
    bal_acc_score = bal_acc(y_true, y_pred)
    print('ACC: %.4f' % acc_score)
    print('Bal. ACC: %.4f' % bal_acc_score)
    return acc_score, bal_acc_score

def calculate_f1(y_true, y_pred, average = 'binary'):
    f1_metric_score = f1(y_true, y_pred, average = average)
    print('F1: %.4f' % f1_metric_score)
    return f1_metric_score

def generate_classification_report(y_true, y_pred, target_names):
    print(classification_report(y_true, y_pred, target_names = target_names))

def visualize_model(model, feature_names, class_names):
    plt.figure(figsize=(25, 8))
    tree.plot_tree(model, feature_names=feature_names, class_names=class_names)
    plt.show()

def visualize_model_dtreeviz(model, X_set, y_set, feature_names, class_names):
    viz = dtreeviz.model(
        model,
        X_set, y_set,
        feature_names=feature_names,
        class_names=list(class_names)
    )
    viz.view(orientation='LR', show_node_labels=True)
    
def save_model(model, filename):
    joblib.dump(model, filename)

def load_model(filename):
    return joblib.load(filename)


In [ ]:
def visualize_feature_importance(model, feature_names):
    feature_importance = model.feature_importances_
    feature_names = np.array(feature_names)
    sorted_feature_names = np.argsort(feature_importance)

    ypos = np.arange(len(feature_names))

    fig= plt.figure(figsize=(8,4))
    plt.barh(ypos, feature_importance[sorted_feature_names])
    plt.xlim([0,1])
    plt.ylabel('Features')
    plt.xlabel('Importance')
    plt.yticks(ypos,feature_names[sorted_feature_names] );
    plt.title('Decision Tree Feature Importance')
    
def calculate_pruning_path(X_set, y_set):
    path = DecisionTreeClassifier().cost_complexity_pruning_path(X_set, y_set)
    ccp_alphas, impurities = path.ccp_alphas, path.impurities
    return ccp_alphas, impurities

def tune_model(X_set, y_set, ccp_alphas):
#     ccp_alphas, impurities = calculate_pruning_path(X_set, y_set)
    parameters = {'ccp_alpha': ccp_alphas[:-1]}
    clftree = DecisionTreeClassifier()
    clfsearch = GridSearchCV(clftree, parameters, cv = StratifiedKFold(n_splits = 5, shuffle = True, random_state = 42), verbose = 2)
    clfsearch.fit(X_set, y_set)
    best_model = clfsearch.best_estimator_
    best_model.fit(X_set, y_set)
    return best_model

def visualize_decision_tree_instance(model, X_set, y_set, x, feature_names, class_names):
    viz = dtreeviz.model(
        model,
        X_train, y_train,
        feature_names=feature_names,
        class_names=list(class_names),
        X=x
    )
    return viz.view(orientation='LR', show_node_labels=True)

In [ ]:
target_labels, encoded_targets = encode_targets(diagnoses)
X_train, X_val, y_train, y_val = split_dataset(sampled_features, diagnoses)
dtree_model = train_model(X_train, y_train)
cross_validation(sampled_features, diagnoses, dtree_model)
predictions = test_model(X_val, y_val, dtree_model)
calculate_accuracies(y_val, predictions)
calculate_f1(y_val, predictions)
generate_classification_report(y_val, predictions, target_labels)

In [ ]:
visualize_model(dtree_model, features, target_labels)

In [ ]:
viz = dtreeviz.model(dtree_model,
                    X_train, y_train,
                    feature_names = features,
                    class_names = list(target_labels))

viz.view(orientation = 'LR',
        show_node_labels = True)

In [ ]:
visualize_feature_importance(dtree_model, features)

In [ ]:
ccp_alphas, impurities = calculate_pruning_path(sampled_features, diagnoses)
dtree_best = tune_model(X_train, y_train, ccp_alphas)
dtree_best

In [ ]:
cross_validation(sampled_features, diagnoses, dtree_best)

In [ ]:
dtree_best.fit(X_train, y_train)

In [ ]:
predictions = test_model(X_val, y_val, dtree_best)
calculate_accuracies(y_val, predictions)
calculate_f1(y_val, predictions)
generate_classification_report(y_val, predictions, target_labels)

In [ ]:
visualize_model(dtree_best, features, target_labels)

The cell below keeps giving me issues so I've commented it out for now

In [ ]:
# value = np.random.randint(0, len(X_val))

# x = X_val[value, :]

# print('True Value')
# print(target_labels[y_val[value]])

# viz = dtreeviz.model(dtree_best,
#                     X_train, y_train,
#                     X = x,
#                     feature_names = features,
#                     class_names = list(target_labels))

# viz.view(orientation = 'LR',
#         show_node_labels = True)

In [ ]:
visualize_feature_importance(dtree_best, features)

### TODO: Remove Everything below